# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime

# Import API key
from api_keys import weather_api_key as w_api

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [15]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

637

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [16]:
found_city = []
latitude = []
longitude = []
max_temp = []
humidity = []
cloudiness = []
wind_speed = []
country = []
date = []
record = 0

print('Beginning Data Retrieval')
print('-----------------------------------')

for city in cities:
    
    #Specify base URL and the desired units
    url = "http://api.openweathermap.org/data/2.5/weather?"
    units = "metric"

    # Build partial query URL
    query_url = f"{url}appid={w_api}&units={units}&q=" + city
    
    response = requests.get(query_url)
    response_json = response.json()
    
    try:
        record = record + 1
        found_city.append(response_json['name'])
        latitude.append(response_json['coord']['lat'])
        longitude.append(response_json['coord']['lon'])
        max_temp.append(response_json['main']['temp_max'])
        humidity.append(response_json['main']['humidity'])
        cloudiness.append(response_json['clouds']['all'])
        wind_speed.append(response_json['wind']['speed'])
        country.append(response_json['sys']['country'])
        date.append(datetime.utcfromtimestamp(response_json['dt']).strftime('%Y-%m-%d %H:%M:%S'))
        
        print(f"Processing Record {record}. | {city.title()}")
    except:
        print(f"Record: {record} / City: {city.title()} not found. Proceeding to next record. ")

print('-----------------------------------')
print('Data Retrieval Complete')
print('-----------------------------------')

Beginning Data Retrieval
-----------------------------------
Processing Record 1. | Ahipara
Processing Record 2. | Muros
Processing Record 3. | Ribeira Grande
Processing Record 4. | Marawi
Processing Record 5. | Busselton
Processing Record 6. | Santa Maria
Processing Record 7. | Visnes
Processing Record 8. | Nueve De Julio
Processing Record 9. | Mataura
Processing Record 10. | Kapaa
Processing Record 11. | Esfahan
Processing Record 12. | Cabedelo
Processing Record 13. | Georgetown
Processing Record 14. | Mayo
Processing Record 15. | Los Llanos De Aridane
Processing Record 16. | Severo-Kurilsk
Record: 17 / City: Illoqqortoormiut not found. Proceeding to next record. 
Processing Record 18. | Tuatapere
Processing Record 19. | Nikolskoye
Processing Record 20. | Kodiak
Processing Record 21. | Bredasdorp
Processing Record 22. | Clyde River
Processing Record 23. | Boende
Processing Record 24. | Yellowknife
Processing Record 25. | Todos Santos
Processing Record 26. | Rikitea
Record: 27 / City:

Processing Record 224. | Merauke
Record: 225 / City: Jujuy not found. Proceeding to next record. 
Processing Record 226. | Berlevag
Processing Record 227. | Saskylakh
Processing Record 228. | Flinders
Processing Record 229. | Ballyhaunis
Processing Record 230. | Nioki
Processing Record 231. | Bom Jesus Da Lapa
Processing Record 232. | Victoria
Processing Record 233. | Ilebo
Processing Record 234. | Kavaratti
Processing Record 235. | Sisophon
Processing Record 236. | Sitka
Processing Record 237. | Kuvandyk
Record: 238 / City: Mocambique not found. Proceeding to next record. 
Processing Record 239. | Baisha
Processing Record 240. | Belaya Gora
Processing Record 241. | Isangel
Processing Record 242. | Olafsvik
Processing Record 243. | Coquimbo
Processing Record 244. | Beringovskiy
Processing Record 245. | Astrea
Processing Record 246. | Corlatel
Processing Record 247. | Longyearbyen
Record: 248 / City: Amderma not found. Proceeding to next record. 
Processing Record 249. | Kenai
Processin

Processing Record 448. | Itaituba
Processing Record 449. | Zhigansk
Processing Record 450. | Shumikha
Record: 451 / City: Saleaula not found. Proceeding to next record. 
Processing Record 452. | Wanning
Processing Record 453. | Rosetta
Processing Record 454. | Beatrice
Processing Record 455. | Lahat
Processing Record 456. | Soyo
Processing Record 457. | Khasan
Processing Record 458. | Lamar
Processing Record 459. | Hasaki
Processing Record 460. | Aksarka
Processing Record 461. | Collinsville
Processing Record 462. | Port Elizabeth
Processing Record 463. | Kruisfontein
Processing Record 464. | Westport
Processing Record 465. | Riyadh
Processing Record 466. | Kangaatsiaq
Processing Record 467. | Tigil
Processing Record 468. | Owerri
Processing Record 469. | Aqtobe
Processing Record 470. | Lira
Processing Record 471. | Kommunisticheskiy
Record: 472 / City: Sentyabrskiy not found. Proceeding to next record. 
Processing Record 473. | San Francisco
Processing Record 474. | Shingu
Processing 

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [14]:
len(date)

576

In [18]:
city_weather = pd.DataFrame({'City': found_city,
                             'Latitude': latitude,
                             'Longitude': longitude,
                             'Max Temperature (degC)': max_temp,
                             'Humidity (%)': humidity,
                             'Cloudiness (%)': cloudiness,
                             'Wind Speed (m/s)': wind_speed,
                             'Country': country,
                             "Date/Time (UTC)": date})



## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


In [ ]:
# Extract relevant fields from the data frame


# Export the City_Data into a csv


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots

In [ ]:
# Create Northern and Southern Hemisphere DataFrames

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression